# Space Invaders

## Dependencies

In [1]:
!pip install tensorflow==2.9.1 gym keras-rl2 gym[atari]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


ROM instructions: https://github.com/openai/atari-py#roms

In [2]:
!python -m atari_py.import_roms roms

copying space_invaders.bin from roms/Space Invaders.bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/space_invaders.bin


## Exploration and baseline

In [3]:
import gym
import random
import numpy as np

In [4]:
env = gym.make("SpaceInvaders-v4")
print(env.observation_space.shape)

(210, 160, 3)


In [5]:
env.unwrapped.get_action_meanings()

['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']

In [80]:
EPISODES = 100
scores = []
for episode in range(1, EPISODES + 1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        # env.render()
        action = random.choice(range(env.action_space.n))
        n_state, reward, done, info = env.step(action)
        score += reward
    
    scores.append(score)
    print(f"Episode {episode}: Reward == {score}")

avg = np.mean(scores)
print(f"Average reward: {avg}")
env.close()

Episode 1: Reward == 105.0
Episode 2: Reward == 110.0
Episode 3: Reward == 210.0
Episode 4: Reward == 265.0
Episode 5: Reward == 110.0
Episode 6: Reward == 45.0
Episode 7: Reward == 35.0
Episode 8: Reward == 180.0
Episode 9: Reward == 210.0
Episode 10: Reward == 315.0
Episode 11: Reward == 110.0
Episode 12: Reward == 120.0
Episode 13: Reward == 110.0
Episode 14: Reward == 215.0
Episode 15: Reward == 105.0
Episode 16: Reward == 335.0
Episode 17: Reward == 180.0
Episode 18: Reward == 360.0
Episode 19: Reward == 155.0
Episode 20: Reward == 125.0
Episode 21: Reward == 65.0
Episode 22: Reward == 125.0
Episode 23: Reward == 110.0
Episode 24: Reward == 105.0
Episode 25: Reward == 80.0
Episode 26: Reward == 120.0
Episode 27: Reward == 155.0
Episode 28: Reward == 115.0
Episode 29: Reward == 35.0
Episode 30: Reward == 35.0
Episode 31: Reward == 185.0
Episode 32: Reward == 105.0
Episode 33: Reward == 55.0
Episode 34: Reward == 60.0
Episode 35: Reward == 75.0
Episode 36: Reward == 135.0
Episode 37

So the baseline is around 150.

## Model

In [18]:
from tensorflow.keras import Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Convolution2D, Resizing, Rescaling
from tensorflow.python.keras.layers.core import Lambda
from tensorflow.keras.optimizers import Adam
from tensorflow.image import rgb_to_grayscale

In [30]:
def build_model(height, width, channels, actions):
    model = Sequential()
    model.add(Input(shape=(height, width, channels)))
    model.add(Lambda(rgb_to_grayscale))
    model.add(Resizing(height // 2, width // 2))
    model.add(Rescaling(1./255)) # normalize to [0, 1]
    model.add(Convolution2D(32, (8,8), strides=(4,4), activation='relu'))
    model.add(Convolution2D(64, (4,4), strides=(2,2), activation='relu'))
    model.add(Convolution2D(64, (3,3), activation='relu'))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    # model.add(Dense(256, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [31]:
height, width, channels = env.observation_space.shape
actions = env.action_space.n

In [32]:
model = build_model(height, width, channels, actions)

In [33]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 module_wrapper_3 (ModuleWra  (None, 210, 160, 1)      0         
 pper)                                                           
                                                                 
 resizing_3 (Resizing)       (None, 105, 80, 1)        0         
                                                                 
 rescaling_3 (Rescaling)     (None, 105, 80, 1)        0         
                                                                 
 conv2d_9 (Conv2D)           (None, 25, 19, 32)        2080      
                                                                 
 conv2d_10 (Conv2D)          (None, 11, 8, 64)         32832     
                                                                 
 conv2d_11 (Conv2D)          (None, 9, 6, 64)          36928     
                                                      

## Agent

In [34]:
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

In [35]:
def build_agent(model, actions):
    policy = LinearAnnealedPolicy(
        EpsGreedyQPolicy(), 
        attr='eps', 
        value_max=1.0, 
        value_min=0.1, 
        value_test=0.2, 
        nb_steps=10000
    )
    memory = SequentialMemory(
        limit=1000, 
        window_length=3
    )
    dqn = DQNAgent(
        model=model, 
        memory=memory, 
        policy=policy,
        enable_dueling_network=True, 
        dueling_type='avg', 
        nb_actions=actions, 
        nb_steps_warmup=1000
    )
    return dqn

In [36]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-4))

/usr/local/lib/python3.7/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


NotImplementedError: ignored

## Train

In [ ]:
dqn.fit(env, nb_steps=10000, visualize=False, verbose=2)

Training for 10000 steps ...
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
  703/10000: episode: 1, duration: 26.704s, episode steps: 703, steps per second:  26, episode reward: 45.000, mean reward:  0.064 [ 0.000, 15.000], mean action: 2.563 [0.000, 5.000],  loss: --, mean_q: --, mean_eps: --
 1372/10000: episode: 2, duration: 666.274s, episode steps: 669, steps per second:   1, episode reward: 100.000, mean reward:  0.149 [ 0.000, 20.000], mean action: 2.508 [0.000, 5.000],  loss: 9.782588, mean_q: 6.822987, mean_eps: 0.893260
 2402/10000: episode: 3, duration: 1772.951s, episode steps: 1030, steps per second:   1, episode reward: 380.000, mean reward:  0.369 [ 0.000, 200.000], mean action: 2.488 [0.000, 5.000],  loss: 6.755573, mean_q: 7.143362, mean_eps: 0.830215
 3062/10000: episode: 4, duration: 1135.876s, episode steps: 660, steps per second:   1, episode reward: 110.000, mean reward:  0.167 [ 0.000, 30.000],

In [ ]:
scores = dqn.test(env, nb_episodes=20, visualize=False)
np.mean(scores.history["episode_reward"])

Testing for 20 episodes ...
Episode 1: reward: 90.000, steps: 442
Episode 2: reward: 100.000, steps: 683
Episode 3: reward: 135.000, steps: 498
Episode 4: reward: 105.000, steps: 559
Episode 5: reward: 195.000, steps: 929
Episode 6: reward: 235.000, steps: 922
Episode 7: reward: 20.000, steps: 380
Episode 8: reward: 75.000, steps: 437
Episode 9: reward: 80.000, steps: 501
Episode 10: reward: 165.000, steps: 663
Episode 11: reward: 105.000, steps: 605
Episode 12: reward: 135.000, steps: 582
Episode 13: reward: 55.000, steps: 458
Episode 14: reward: 60.000, steps: 680
Episode 15: reward: 105.000, steps: 711
Episode 16: reward: 105.000, steps: 509
Episode 17: reward: 170.000, steps: 924
Episode 18: reward: 160.000, steps: 700
Episode 19: reward: 220.000, steps: 1154
Episode 20: reward: 50.000, steps: 380


118.25